https://www.kaggle.com/code/mirenaborisova/srrnaf-8/notebook

https://www.kaggle.com/code/pranshubahadur/esm2-rmdb-rna-dataset

In [2]:
import warnings
warnings.simplefilter('ignore')

import pandas as pd
pd.set_option('display.max_columns', 30)

In [3]:
import tensorflow as tf

resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.experimental.TPUStrategy(resolver)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local


2023-12-30 11:39:31.792877: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-30 11:39:31.792984: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-30 11:39:31.793064: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-30 11:39:31.793151: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-30 11:39:31.793218: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-30 11:39:31.793436: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-30 11:39:31.793525: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority un

INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:6, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


In [4]:
rmdb = pd.read_csv('/kaggle/input/rmdb-rna-mapping-database-2023-data/rmdb_data.v1.3.0.csv')

In [5]:
rmdb_SN_filter = rmdb[rmdb.SN_filter == 1]
error_feats = [feat for feat in rmdb_SN_filter.columns if 'error' in feat]
rmdb_SN_filter_no_error = rmdb_SN_filter.drop(columns = error_feats)


In [6]:
# experiment_type = rmdb_SN_filter_no_error.experiment_type.unique()

# rmdb_etype_2A3 = [experiment_type[0]]
# rmdb_etype_DMS = (experiment_type[1:]).tolist()

In [7]:
rmdb_etype_2A3 = ['1M7','NMIA','BzCN']
rmdb_etype_DMS = ['BzCN_cotx', 'DMS_cotx', 'DMS_M2_seq', 'DMS']

In [8]:
train_quick = pd.read_csv('/kaggle/input/stanford-ribonanza-rna-folding/train_data_QUICK_START.csv')
train_quick_2A3 = train_quick[train_quick.experiment_type == '2A3_MaP'].reset_index(drop=True)
train_quick_DMS = train_quick[train_quick.experiment_type == 'DMS_MaP'].reset_index(drop=True)

In [9]:
rmdb_2A3 = rmdb_SN_filter_no_error[rmdb_SN_filter_no_error.experiment_type.isin(rmdb_etype_2A3)].reset_index(drop=True)
rmdb_2A3 = pd.concat([train_quick_2A3, rmdb_2A3]).reset_index(drop=True)

rmdb_DMS = rmdb_SN_filter_no_error[rmdb_SN_filter_no_error.experiment_type.isin(rmdb_etype_DMS)].reset_index(drop=True)
rmdb_DMS = pd.concat([train_quick_DMS, rmdb_DMS]).reset_index(drop=True)

In [10]:
X_2A3 = rmdb_2A3.sequence
rmdb_2A3 = rmdb_2A3.filter(regex='reactivity_[0-9]')
rmdb_2A3 = rmdb_2A3.fillna(0)
X_2A3 = X_2A3.iloc[rmdb_2A3.index].reset_index(drop=True)
Y_2A3 = rmdb_2A3.reset_index(drop=True)

X_DMS = rmdb_DMS.sequence
rmdb_DMS = rmdb_DMS.filter(regex='reactivity_[0-9]')
rmdb_DMS = rmdb_DMS.fillna(0)
X_DMS = X_DMS.iloc[rmdb_DMS.index].reset_index(drop=True)
Y_DMS = rmdb_DMS.reset_index(drop=True)


In [11]:
import gc

del train_quick
del train_quick_2A3
del train_quick_DMS

del rmdb
del rmdb_SN_filter
del error_feats
del rmdb_SN_filter_no_error
del rmdb_etype_2A3
del rmdb_etype_DMS

gc.collect()


78

In [12]:
from tensorflow.keras import Model
from transformers import TFAutoModel

class RNA(Model):
    
    def __init__(self):
        
        super().__init__()
        self.encoder = TFAutoModel.from_pretrained('AmelieSchreiber/esm2_t6_8M_UR50D_rna_binding_site_predictor')
        self.dropout = tf.keras.layers.Dropout(0.2)
        self.dense = tf.keras.layers.Dense(1)
        
    def call(self, x):
        
        x = self.encoder(x).last_hidden_state
        
        return tf.squeeze(self.dense(x), -1)
    


In [13]:
def loss(X, Y):
    
    X_mask = tf.math.is_nan(X)
    X = tf.where(X_mask, tf.zeros_like(X), X)
    sum_mask = tf.math.reduce_sum(tf.where(X_mask, tf.zeros_like(X), tf.ones_like(X)))
    loss = tf.math.abs(X - Y)
    loss = tf.where(X_mask, tf.zeros_like(loss), loss)
    loss = tf.math.reduce_sum(loss) / (sum_mask if sum_mask != 0.0 else 1.0)
    
    return loss


In [14]:
text_vectorization = tf.keras.layers.TextVectorization(output_mode='int',
                                                       ngrams=1,
                                                       output_sequence_length=Y_DMS.shape[1],
                                                       split='character',
                                                       vocabulary=['a', 'c', 'g', 'u'])

In [15]:
with strategy.scope():
    
    model_DMS = RNA()
    model_DMS.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-4), loss=loss)
    
    train_ds = tf.data.Dataset.from_tensor_slices((X_DMS.values, Y_DMS.values)) \
        .batch(128) \
        .map(lambda x, y: (text_vectorization(x), tf.clip_by_value(y, 0, 1)))
    train_ds = train_ds.shuffle(train_ds.cardinality())
    train_data = train_ds.take(int(len(train_ds) * 0.8))
    validation_data = train_ds.skip(int(len(train_ds) * 0.8)).take(int(len(train_ds) * 0.2))

    model_DMS.fit(train_data, validation_data=validation_data, epochs=25, batch_size=128)

2023-12-30 11:40:37.996597: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-30 11:40:37.996747: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFEsmModel: ['classifier.bias', 'esm.embeddings.position_embeddings.weight', 'classifier.weight']
- This IS expected if you are initializing TFEsmModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFEsmModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFEsmModel were not initialized from 

Epoch 1/25


2023-12-30 11:41:07.666959: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Adam/AssignAddVariableOp.
2023-12-30 11:41:16.513147: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-30 11:41:16.516452: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-30 11:41:16.516570: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-30 11:41:16.516709: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-30 11:41:16.516831: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-30 11:41:16.517115: E ./tensorflow/compiler/xla/stream_executor/st

   1/1204 [..............................] - ETA: 17:10:30 - loss: 0.7443

2023-12-30 11:41:38.564979: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-30 11:41:38.565509: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-30 11:41:38.565587: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-30 11:41:38.565771: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-30 11:41:38.565867: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-30 11:41:38.566340: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-30 11:41:38.568576: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority un

...

2023-12-30 11:41:53.746902: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.


 215/1204 [====>.........................] - ETA: 1:56 - loss: 0.1326

2023-12-30 11:42:03.850116: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.


1145/1204 [===========================>..] - ETA: 6s - loss: 0.0830

KeyboardInterrupt: 

In [18]:
text_vectorization_preds = tf.keras.layers.TextVectorization(output_mode='int',
                                                             ngrams=1,
                                                             output_sequence_length=457,
                                                             split='character',
                                                             vocabulary=['a', 'c', 'g', 'u'])

In [ ]:
test = pd.read_csv('/kaggle/input/stanford-ribonanza-rna-folding/test_sequences.csv')


In [ ]:
with strategy.scope():
    
    preds_DMS = model_DMS.predict(tf.data.Dataset.from_tensor_slices((test.sequence)) \
        .batch(128) \
        .map(lambda x: text_vectorization_preds(x)))
    

In [ ]:
# df_DMS = pd.DataFrame(preds_DMS)
# df_DMS.to_csv('preds_DMS.csv')


In [ ]:
with strategy.scope():
    
    model_2A3 = RNA()
    model_2A3.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-4), loss=loss)
    
    train_ds = tf.data.Dataset.from_tensor_slices((X_2A3.values, Y_2A3.values)) \
        .batch(128) \
        .map(lambda x, y: (text_vectorization(x), tf.clip_by_value(y, 0, 1)))
    train_ds = train_ds.shuffle(train_ds.cardinality())
    train_data = train_ds.take(int(len(train_ds) * 0.8))
    validation_data = train_ds.skip(int(len(train_ds) * 0.8)).take(int(len(train_ds) * 0.2))

    model_2A3.fit(train_data, validation_data=validation_data, epochs=25, batch_size=128)
    

In [ ]:
with strategy.scope():
    
    preds_2A3 = model_2A3.predict(tf.data.Dataset.from_tensor_slices((test.sequence)) \
        .batch(128) \
        .map(lambda x: text_vectorization_preds(x)))
    

In [ ]:
# df_2A3 = pd.DataFrame(preds_2A3)
# df_2A3.to_csv('preds_2A3.csv')

In [ ]:
test_seq_lengths = test.sequence.str.len()


In [ ]:
import numpy as np

preds_2A3_bylength = []

for i in range(test_seq_lengths.size):
    
    x = np.reshape(preds_2A3[i, :test_seq_lengths[i]], (-1, 1))
    x = np.clip(x, 0, 1)
    
    preds_2A3_bylength.append(x)

In [ ]:
preds_2A3 = np.concatenate(preds_2A3_bylength, 0)

In [ ]:
preds_DMS_bylength = []

for i in range(test_seq_lengths.size):
    
    x = np.reshape(preds_DMS[i, :test_seq_lengths[i]], (-1, 1))
    x = np.clip(x, 0, 1)
    
    preds_DMS_bylength.append(x)

In [ ]:
preds_DMS = np.concatenate(preds_DMS_bylength, 0)

In [ ]:
preds_DMS.shape[0]


In [ ]:
submission = pd.DataFrame({'id': np.arange(0, preds_DMS.shape[0], 1),
                           'reactivity_DMS_MaP': preds_DMS[:, 0],
                           'reactivity_2A3_MaP': preds_2A3[:, 0]})


In [ ]:
submission.head()

In [ ]:
!pip install pyarrow

In [ ]:
submission.to_parquet('submission.parquet', index=False)